<a href="https://colab.research.google.com/github/Dhaneshkp/DesktopAssistant/blob/main/DATA_processing_expConv2Dv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import numpy as np

# Set a random seed for reproducibility
np.random.seed(42)

# Parameters
n_samples = 10000
unique_risk_classes = ['A', 'B', 'C', 'D']
n_anomalies = 200
# Generate random ages, durations, and premiums
ages = np.random.randint(18, 70, size=(n_samples, 1))
durations = np.random.randint(1, 20, size=(n_samples, 1))
premiums = ages * durations + np.random.normal(0, 50, size=(n_samples, 1))

# Define risk class multipliers
riskclass_multipliers = {
    'A': 1.05,
    'B': 1.1,
    'C': 1.3,
    'D': 1.0
}

# Initialize the result array
result_array = np.empty((n_samples, 1, 3, len(unique_risk_classes)))

# Iterate over samples and risk classes
for i in range(n_samples):
    for j, risk_class in enumerate(unique_risk_classes):
        # Apply risk class multiplier to premium
        premium_with_risk = premiums[i] * riskclass_multipliers[risk_class]

        # Assign age, duration, and premium to the result array
        result_array[i, 0, :, j] = np.concatenate((ages[i], durations[i], premium_with_risk))

anomaly_indices = np.random.choice(n_samples * len(unique_risk_classes), n_anomalies, replace=False)
for idx in anomaly_indices:
    sample_idx = idx // len(unique_risk_classes)
    risk_class_idx = idx % len(unique_risk_classes)
    result_array[sample_idx, 0, 2, risk_class_idx] += np.random.randint(500, 1000)  # Add a large random value to the premium

print(result_array.shape)  # Output: (10000, 1, 3, 4)
print(result_array)


(10000, 1, 3, 4)
[[[[5.60000000e+01 5.60000000e+01 5.60000000e+01 5.60000000e+01]
   [1.10000000e+01 1.10000000e+01 1.10000000e+01 1.10000000e+01]
   [6.34330506e+02 6.64536721e+02 7.85361579e+02 6.04124292e+02]]]


 [[[6.90000000e+01 6.90000000e+01 6.90000000e+01 6.90000000e+01]
   [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00]
   [6.28988242e+01 6.58940063e+01 7.78747348e+01 5.99036421e+01]]]


 [[[4.60000000e+01 4.60000000e+01 4.60000000e+01 4.60000000e+01]
   [1.80000000e+01 1.80000000e+01 1.80000000e+01 1.80000000e+01]
   [9.26381249e+02 9.70494642e+02 1.14694821e+03 8.82267856e+02]]]


 ...


 [[[5.70000000e+01 5.70000000e+01 5.70000000e+01 5.70000000e+01]
   [1.80000000e+01 1.80000000e+01 1.80000000e+01 1.80000000e+01]
   [1.08891627e+03 1.14076943e+03 1.34818205e+03 1.03706311e+03]]]


 [[[6.40000000e+01 6.40000000e+01 6.40000000e+01 6.40000000e+01]
   [1.80000000e+01 1.80000000e+01 1.80000000e+01 1.80000000e+01]
   [1.19011274e+03 1.24678478e+03 1.47347292e+03 1

In [ ]:

# Split the data into training and test sets
# Split the data into training and test sets
X_train, X_test, train_indices, test_indices = train_test_split(
    result_array,
    np.arange(result_array.shape[0]),  # Create indices for the array
    test_size=0.2,
    random_state=42
)
#train_indices, test_indices = train_test_split(data_reshaped.index, test_size=0.2, random_state=42)
# Define the CNN architecture
input_shape = (X_train.shape[1], X_train.shape[2],X_train.shape[3])    # Assuming X_train is a 3D array
input_layer = Input(shape=input_shape)
# Define the CNN architecture
input_shape = (X_train.shape[1], X_train.shape[2],X_train.shape[3])    # Assuming X_train is a 3D array
input_layer = Input(shape=input_shape)

conv_layer = Conv2D(32, (1,3), activation="relu",padding='same')(input_layer)
conv_layer = MaxPooling2D(pool_size=(1,1))(conv_layer)

output_shape = (conv_layer.shape[1], conv_layer.shape[2], conv_layer.shape[3])  # Adjust as needed

# Reshape the output of the convolutional layers
reshape_layer = Reshape(output_shape)(conv_layer)

# Use transposed convolution (Conv2DTranspose) to upsample and reconstruct
deconv_layer = Conv2DTranspose(32, (1, 3), activation='relu', padding='same')(reshape_layer)
output_layer = Conv2D(X_train.shape[3], (1, 1), activation='linear', padding='same')(deconv_layer)  # Output 4 channels

cnn_model = Model(inputs=input_layer, outputs=output_layer)
cnn_model.compile(optimizer='adam', loss='mse')

# ... (rest of your code for training and prediction)

cnn_model = Model(inputs=input_layer, outputs=output_layer)
cnn_model.compile(optimizer='adam', loss='mse')

# Train the model
history = cnn_model.fit(X_train, X_train,
                        epochs=50,
                        batch_size=32,
                        validation_split=0.1,
                        verbose=1)

# Predict the training data
pred_train = cnn_model.predict(X_train)
mse_train = np.mean(np.power(X_train - pred_train,2), axis=1)



Epoch 1/50
1069/1069 [==============================] - 5s 4ms/step - loss: 9.8263 - val_loss: 0.0941
Epoch 2/50
1069/1069 [==============================] - 3s 3ms/step - loss: 0.0574 - val_loss: 0.0533
Epoch 3/50
1069/1069 [==============================] - 3s 2ms/step - loss: 0.0393 - val_loss: 0.0343
Epoch 4/50
1069/1069 [==============================] - 2s 2ms/step - loss: 0.0257 - val_loss: 0.0298
Epoch 5/50
1069/1069 [==============================] - 4s 3ms/step - loss: 0.0183 - val_loss: 0.0236
Epoch 6/50
1069/1069 [==============================] - 4s 4ms/step - loss: 0.0235 - val_loss: 0.0212
Epoch 7/50
1069/1069 [==============================] - 3s 3ms/step - loss: 0.0148 - val_loss: 0.0177
Epoch 8/50
1069/1069 [==============================] - 3s 2ms/step - loss: 0.0078 - val_loss: 0.0179
Epoch 9/50
1069/1069 [==============================] - 2s 2ms/step - loss: 0.0133 - val_loss: 0.0171
Epoch 10/50
1069/1069 [==============================] - 3s 3ms/step - loss: 0.006

In [ ]:
pred_train

array([[[[31.972307  , 40.00123   , 34.051006  , 36.973846  ],
         [ 2.9870317 ,  1.9752192 ,  3.9910448 ,  2.9981515 ],
         [ 3.3882978 ,  3.9726245 ,  6.6289487 ,  5.53451   ]]],


       [[[28.005468  , 32.9821    , 28.97913   , 31.020714  ],
         [ 3.9718254 ,  2.9896245 ,  8.000642  ,  5.0119658 ],
         [ 6.0242825 ,  5.380406  ,  5.4330883 ,  6.232376  ]]],


       [[[26.98937   , 31.993385  , 29.000933  , 30.012524  ],
         [ 6.979805  ,  1.973206  ,  1.0003712 ,  5.005399  ],
         [ 4.425792  ,  2.1919494 ,  4.79101   ,  4.2905316 ]]],


       ...,


       [[[36.01559   , 46.983826  , 38.99002   , 43.00821   ],
         [ 2.9846003 ,  6.984909  ,  4.0057    ,  2.0082014 ],
         [ 4.854017  , 20.876736  ,  4.537457  ,  5.2485304 ]]],


       [[[19.024096  , 19.006693  , 19.014526  , 18.988003  ],
         [ 0.99281967,  5.9947777 ,  3.0060856 ,  3.9952428 ],
         [ 2.2949164 , 11.452527  ,  2.9816148 ,  9.600874  ]]],


       [[[26.972803  

In [ ]:

# Predict the test data
pred_test = cnn_model.predict(X_test)
mse_test = np.mean(np.power(X_test - pred_test  , 2), axis=1)

# Define a threshold for anomaly detection
threshold = np.percentile(mse_train, 95)

# Identify anomalies in the test set
anomalies = mse_test > threshold

# Print the results
print(f"Threshold: {threshold}")
print(f"Number of anomalies in the test set: {np.sum(anomalies)}")

X_test
pred_test

297/297 [==============================] - 1s 2ms/step
Threshold: 0.001513257753003927
Number of anomalies in the test set: 5586


array([[[[30.006018  , 35.983788  , 31.986225  , 34.01788   ],
         [ 3.9814904 ,  6.9824476 ,  3.0026877 ,  4.0104017 ],
         [ 3.880396  ,  8.350938  ,  2.6971538 ,  4.4835405 ]]],


       [[[23.984531  , 26.979704  , 24.985565  , 26.032402  ],
         [ 7.9688425 ,  0.9660402 ,  5.049216  ,  1.9527314 ],
         [ 8.715238  ,  1.8196716 ,  3.1029289 ,  1.7817905 ]]],


       [[[37.038433  , 50.98192   , 40.95598   , 45.025665  ],
         [ 5.9813952 ,  3.9833617 ,  3.0017512 ,  5.0155087 ],
         [ 7.8668957 , 22.75681   ,  6.08524   , 13.917496  ]]],


       ...,


       [[[22.013504  , 24.004747  , 22.005016  , 22.99803   ],
         [ 0.9931549 ,  0.9856997 ,  5.9866147 ,  2.985222  ],
         [ 1.9236456 ,  1.6129543 ,  8.497366  ,  6.724445  ]]],


       [[[33.002647  , 42.989616  , 36.005573  , 38.995117  ],
         [ 8.987646  ,  3.9812865 ,  5.9868164 ,  8.006377  ],
         [ 8.048571  ,  8.821319  ,  7.307101  , 12.701252  ]]],


       [[[30.002256  

In [ ]:
# Create a copy of the original test DataFrame
test_df_with_anomalies = filtered_df_encoded.iloc[test_indices].copy()

# Add an 'anomaly' column to the test DataFrame
test_df_with_anomalies['anomaly'] = anomalies

print(f"Threshold: {threshold}")
print(f"Number of anomalies in the test set: {np.sum(anomalies)}")

# Display the test DataFrame with anomaly tags
test_df_with_anomalies

Threshold: 0.001513257753003927
Number of anomalies in the test set: 5586


ValueError: Shape of passed values is (9, 6, 4), indices imply (10, 9)

ValueError: Shape of passed values is (9, 6, 4), indices imply (10, 9)

In [ ]:
# prompt: crate an isolation forest to detect anomalies in teh result_df_encoded and fina the index of anomalies in teh data and crate a data frame like test_df_with_anomalies , use the xtrain and xtest etc using the xtrain and xtest indices created in the previuos code, and for isolation forest i think we need y test and train so creat theam , if we drop premium rate then thas sdropiing the ioput

import numpy as np
import pandas as pd
from sklearn.ensemble import IsolationForest
from sklearn.model_selection import train_test_split

# Assuming 'result_df_encoded' is your DataFrame from the preceding code

# Prepare the data for Isolation Forest
X = result_df_encoded.drop('Premium_rate', axis=1).values  # Assuming 'Premium_rate' is the target variable

# Split the data into training and test sets (using the same indices as before)
X_train = X[train_indices]
X_test = X[test_indices]
y_train = result_df_encoded.loc[train_indices, 'Premium_rate'].values
y_test = result_df_encoded.loc[test_indices, 'Premium_rate'].values

# Create an Isolation Forest model
model = IsolationForest(contamination=0.05, random_state=42)  # Adjust contamination as needed

# Fit the model to the training data
model.fit(X_train)

# Predict anomalies on the test data
y_pred = model.predict(X_test)

# Identify anomalies (-1 indicates an anomaly)
anomalies = y_pred == -1

# Create a copy of the original test DataFrame
test_df_with_anomalies_if = result_df_encoded.iloc[test_indices].copy()

# Add an 'anomaly' column to the test DataFrame
test_df_with_anomalies_if['anomaly'] = anomalies

# Print the results
print(f"Number of anomalies in the test set (Isolation Forest): {np.sum(anomalies)}")

# Display the test DataFrame with anomaly tags
test_df_with_anomalies_if


Number of anomalies in the test set (Isolation Forest): 455


,Age,Duration,Premium_rate,level_2_Female_NP2,level_2_Female_NS2,level_2_Male_NP2,level_2_Male_NS2,anomaly
21185,22.0,4.0,8.359788,0.0,0.0,1.0,0.0,False
21081,41.0,2.0,4.408889,1.0,0.0,0.0,0.0,False
38072,35.0,9.0,9.813333,0.0,0.0,0.0,1.0,False
37127,21.0,9.0,7.733333,0.0,0.0,0.0,1.0,False
22786,38.0,3.0,5.911330,0.0,0.0,0.0,1.0,False
...,...,...,...,...,...,...,...,...
25543,42.0,3.0,8.059259,0.0,0.0,1.0,0.0,False
23315,28.0,5.0,7.714286,0.0,0.0,1.0,0.0,False
8920,41.0,7.0,12.370370,0.0,0.0,1.0,0.0,False
21822,47.0,5.0,15.828571,0.0,0.0,1.0,0.0,False
